#Modelling Rabi Oscillations in Three Level Optical Systems

The following document outlines the theory behind and implements the procedure for modelling **Rabi oscillations** in the excitation probability of a **biexciton-exciton** scheme invoked in a **quantum dot** excited by a laser. This model will make use of **QuTiP**, an python SDK written for modelling open quantum systems.

For a more thorough explaination of the theory outlined in this document and the decisions made to optimize this simulation, see this document.

In [ ]:
!pip install qutip

This project will take advantage of QuTiP's built in Master Equation Solver, which is equipt to solve a given **Lindblad master equation** of the form:

$$\rho ̇(t) = {-i\over \hbar}[H(t),\rho(t)] + \sum {1 \over 2}[2A_n\rho (t)A_n^† - \rho(t)A_n^†A_n - A_n^†A_n\rho (t)]$$

where $A_n$ are collapse operators and $\rho(t)$ is the density matrix of the system at t = 0 (implying $\rho ̇$ is the time evolved system) [1].

In order to see Rabi Oscillations in the aforemention excitation scheme, the system must be subject to laser pulses
 of varying intensity $\Omega_{max}$. In short, it is expected this range of frequenies, when a constant pulse length is held, will invoke a **sinusoidal excitation probability** as $\Omega_{max}$ increases, fluctuating periodically between maxima and minima.

This system, when introduced to the system, will drive the transition between ground state $|g\rangle$, exciton state $|e\rangle$, and biexciton state $|b\rangle$.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import qutip as qt
from qutip import Options, qutrit_basis, dag, fock, about, basis, destroy, mesolve, qeye, sigmaz, tensor

#Parameters and Values
#n = 50
gamma_b = 0.5 #biexciton decay constant (THz)
gamma_x = 1 #exiton decay constant (THz)
globaldt = 0.15/gamma_b
minpointsperpulse = 5.0
tmin = 0.001
tmax = 40*gamma_b

#t_list = np.linspace(0.01, 40*gamma_b, n) #Lifetime of the simulation
t_i = 3.5 #Centre of the Gaussian pulse (offset) (ns)
sigma = 4*gamma_b #Pulse Length (ns) - multiply by gamma?

if sigma/minpointsperpulse > globaldt:
    t_list = np.linspace(tmin, tmax, int((tmax - tmin)/globaldt))
    n = int((tmax - tmin)/globaldt)
    # otherwise, increase resolution inside the pulse only.
else:
    tlist1 = np.linspace(tmin, sigma*t_i*2.0, \
                             int(minpointsperpulse*(t_i*2.0 - tmin)))[:-1]
    tlist2 = np.linspace(sigma*t_i*2.0, tmax, \
                             int((tmax - sigma*t_i*2.0)/globaldt))
    t_list = np.append(tlist1, tlist2)

omega_max = np.linspace(0.001, np.pi, n) #Maximum intensity of the laser pulse

#System states
gnd = fock(3,0)
ex = fock(3,1)
biex = fock(3,2)

In order to accurately translate a pulse adhering to the above paramters, the **Rabi Frequency**  $\Omega_R$ must be calculated and the pulse must be normalized. For this excitation scheme, a **Gaussian pulse** is applied, which allows for accurate intensity variation. From the supplementary material to Ref. [1],

$$\Omega_R = \Omega_{max} e^{{-ln(2)(t-t_i)^2}\over\sigma^2}$$

Thus, a distinct $\Omega_R$ list must be calculated for each constant $\Omega_{max}$ value to be tested.

To normalize, a normalization factor can be calculated for each Gaussian pulse as the inverse of the area beneath the gaussian.

$$F_{norm} = (\int_{0}^{t_f}\Omega_Rdt)^{-1}$$

This factor, when applied to the respective gaussian, will result in a normalized pulse.



In [ ]:
#Frequency
omega_list = []
final_pulse = []

for freq in range(0,n):
  omega = (np.exp(-(np.log(2)*((t_list-t_i)**2))/(sigma**2))) #Rabi frequency
  omega_list.append(omega)
  #final_pulse.append(omega)

for i in range(0,n):
  norm_val = 1/(np.trapz((omega_list[i]), t_list))
  norm = norm_val*omega_max[i]*sigma
  normalized = (omega_list[i]*norm)
  final_pulse.append(normalized)

To properly model the evolution of the system, four **Louvillian operators** must be defined. Two operators ($L_1$ and $L_2$) describe spontaneous photon emission leading to transitions between adjascent states and the other two ($L_3$ and $L_4$) describe dephasing between adjascent states [1].

$$L_1 = {\gamma_b \over 2}D(|b\rangle\langle x|) $$
$$L_2 = {\gamma_x \over 2}D(|x\rangle\langle g|) $$
$$L_3 = {\gamma_x^d \over 2}D(|b\rangle\langle b| - |x\rangle\langle x|)$$
$$L_4 = {\gamma_b^d \over 2}D(|x\rangle\langle x| - |g\rangle\langle g|)$$

Where $\gamma_b$ and $\gamma_x$ are the spontaneous emission rates for the biexiton and exiton, and  $\gamma_x^d$ and $\gamma_b^d$ are the dephasing rates between of the biexiton-exiton and exiton-ground states, respectively [1].

Also, $D = 2A_n\rho (t)A_n^† - \rho(t)A_n^†A_n - A_n^†A_n\rho (t)$ and can be omitted from the operator definition due to the capabilities of the QuTiP master equation solver, which assumes operators will be input in this form.

$\gamma_x^d$ and $\gamma_b^d$ can either be **time dependent** or **constant**. In the case of time dependence, these rates can be calculated in relation to the aforementioned guassian pulse.

$$\gamma_x^d = \gamma_b^d = \gamma_{I_0} \Omega_{div}^{n_p}$$

In this expression, $\Omega_{div} = {\Omega_R \over \Omega_{max}}$ and $\gamma_{I_0}$ is the rate of amplitude dependent dephasing. $n_p$ is a scaling factor that dictates the geometric behaviour of the system. Assuming $n_p = 2$ allows for a linear relationship suitable for this application [1].

In [ ]:
#Collapse Operators
#hard code decay rate
gamma_I0 = 0.05 #Amplitude of intensity-dependent dephasing rate
n_p = 2

#Run 0 edge case
L1 = (gamma_b/2)*(ex*biex.dag()) #biex-ex spontaneous decay
L2 = (gamma_x/2)*(gnd*ex.dag()) #ex-gnd spontaneous decay
L3 = []
L4 = []

#extrapolate gamma values?
for i in (range(0,n)):
  omega_working = final_pulse[i]
  holder3 = []
  holder4 = []
  for j in (range(0,n)):
    coeff = ((gamma_I0*((omega_working[j]/omega_max[i])**n_p))/2)
    # coeff4 = 1/211
    # coeff3 = 1/119
    val3 = coeff*((biex*biex.dag())-(ex*ex.dag())) #biex-ex dephasing
    val4 = coeff*((ex*ex.dag())-(gnd*gnd.dag())) #ex-gnd dephasing
    holder3.append(val3)
    holder4.append(val4)
  L3.append(holder3)
  L4.append(holder4)

There are several ways in which the system Hamiltonian can be written depending on the frame and picture chosen. Optimal results found when the **rotating frame** is considered. The rotating frame assumes a frame of reference that rotates at the laser's frequency, thus suppressing rapidly rotating terms. With this in mind, the system can be described by the following Hamiltonian, as defined in Ref. [2]:
$$H_{sys} = H_0 + H_E(t)$$
where,
$$H_0 = {E_b\over 2} |i\rangle \langle i|$$
and $$H_E(t) = {(\mu E(t))^2\over E_b}(|g\rangle\langle e|+|e\rangle\langle g|)$$

such that $E(t)$ refers to the pulse driving the system, $\mu$ refers to the coupling strength, and $E_b$ refers to the binding energy.

To further simplify, choosing to implement this Hamiltonian in the **interaction picture** allows $H_0$ to be eliminated, since the intermediate step is not necessary in this picture to accurately represent the system.

Once the Hamiltonian has been defined for each Gaussian pulse, the master equation solver can be implemented. This function accepts arguments as **mesolve(Hamiltonian, initial state, time interval, Collapse Operators, Evolution Operators, ...).**

Once implemented, the expectation values for each Hamiltonian (and, thus, each state of the system) can be obtained via **output.expect[$x$]** where $x$ is the index corresponding to the desired set of values, dictated by the order of the evolution operators (ie. if the evolution operators are defined as [exciton, biexciton], the exciton expectation values can be obtained using output.expect[0]).

**Note:** Since QuTiP assumes $\hbar = 1$, $\hbar$ can be omitted from the Hamiltonian definition.

In [ ]:
#Hamiltonian
C = 1 #dipole coupling strength and binding energy
deltax = 2*np.pi*0.335
H_list = []
for i in range(0,n):
  H0 = qeye(3)
  omega_working = final_pulse[i]
  holder = []
  #H_I = (C/2)*(biex*gnd.dag() + gnd*biex.dag()) #fischer
  #for j in range(0,n):
  H_I = ((1/2)*(gnd*ex.dag()+ ex*gnd.dag() + ex*biex.dag() + biex*ex.dag())) + deltax*(ex*ex.dag()) #huber
    #holder.append(H_I)
  #H_list.append(holder)
  H_list.append([H_I,omega_working])

#Evolution
master_output_ex = []
master_output_biex = []

for val in range(0,n):
  H_working = H_list[val]
  L3_working = L3[val]
  L4_working = L4[val]
  holder_ex = []
  holder_biex = []

  output = mesolve(H_working, gnd, t_list, [L1,L2,L3_working,L4_working], [ex*ex.dag(),biex*biex.dag()])

  master_output_ex.append(output.expect[0])
  master_output_biex.append(output.expect[1])

To quickly revisit the theory behind this model, it is expected that the maximum expectation value for each Gaussian pulse will vary periodically. As such, the area beneath the curves formed by these expectation values will also vary periodically. Thus, the system's Rabi oscillations can be obtained via scaled integration, where:

$$P_b(t_f) = \gamma_b \int_0^{t_f} \langle b| \rho(t)|b\rangle dt$$
$$P_x(t_f) = \gamma_x \int_0^{t_f} \langle x| \rho(t)|x\rangle dt$$

denote the occupation probability of the biexciton and exiton, respectively [1].

In [ ]:
int_freq_ex = []
int_freq_biex = []
max_val_ex = []
max_val_biex = []

for vals in range(0,n):
  integration_ex = gamma_x*np.trapz(master_output_ex[vals], t_list)
  max_ex = max(master_output_ex[vals])
  integration_biex = gamma_b*np.trapz(master_output_biex[vals], t_list)
  max_biex = max(master_output_biex[vals])

  int_freq_ex.append(integration_ex)
  int_freq_biex.append(integration_biex)
  max_val_ex.append(max_ex)
  max_val_biex.append(max_biex)

In [ ]:
#plot
g = n-1
print(n)
fig, ax = plt.subplots(figsize=(8,5))
ax.plot(omega_max, master_output_ex[g], 'r')
ax.plot(omega_max, master_output_biex[g], 'b')
#ax.plot(omega_max, final_pulse[g], 'g')
ax.set_xlabel('frequency')
ax.set_ylabel('Occupation probability')
ax.set_title('Vacuum Rabi oscillations')

In [ ]:
val = gamma_b*np.trapz(master_output_biex[n-1],t_list)
print(val)

1.5997036641130988


##References
[1] T. Huber et al., "Coherence and Degree of Time-Bin Entanglement from Quantum Dots," *Physical Review B*, vol. 93, *(20)*, 2016.

[2] K. Fischer et al., "Dynamical Modelling of Pulsed Two-Photon Interference," *New Journal of Physics*, vol. 18, *(11)*, 2017.